# Лабораторная работа 3

Необходимо разработать программу, обеспечивающую кластеризацию новостного потока. Необходимо загрузить новостную ленту, указанную преподавателем, выделив из нее тематические пометы (при наличии). Далее следует векторизовать тексты, провести их кластеризацию и проверить точность кластеризации по тематическим пометам. Также следует выбрать один из кластеров и обучить на нем классификатор. Точность работы классификатора определяется при помощи f1-меры и матрицы ошибок.

In [1]:
import csv
import re
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 8)

In [14]:
%%time
# "date","type","name","title","text" - headers for CSV
with open('./src/cbr-news.csv', 'r', encoding='UTF-8') as NEWS:
    txt_news = csv.reader(NEWS, delimiter=',')
    news = pd.DataFrame(txt_news, columns=["date","type","name","title","text"])

news.head()


CPU times: user 14 ms, sys: 3.16 ms, total: 17.2 ms
Wall time: 16.6 ms


,date,type,name,title,text
0,"﻿""27 апреля 2023""",Новость,Финансовый конгресс Банка России пройдет 6–7 июля,Банк России возобновляет проведение традиционн...,Тема пленарной сессии — структурная перестройк...
1,27 апреля 2023,Новость,Военнослужащие и члены их семей получили более...,С конца сентября 2022 года до 31 марта 2023 го...,"Без учета кредитных каникул, оформленных в свя..."
2,27 апреля 2023,Новость,В 2022 году МФО увеличили кредитование бизнеса...,В прошлом году микрофинансовые организации (МФ...,13% займов было выдано дистанционно. Таким кан...
3,26 апреля 2023,Новость,Пенсионные накопления застрахованы государство...,Предельный размер гарантийного возмещения по д...,"Такой закон одобрил Совет Федерации, и он всту..."
4,26 апреля 2023,Новость,Фальшивок стало меньше: итоги I квартала,В I квартале 2023 года в банковской системе Ро...,С подробной информацией можно ознакомиться в м...
